In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt

import sys
sys.path.append('../')

In [ ]:
from src.d01_data.block_data_api import BlockDataApi
from src.d01_data.student_data_api import StudentDataApi, _block_features, _census_block_column, \
_diversity_index_features

geoid_name = 'geoid'

# Process data
Load data indexed by the `census_block`/`Block`/`Geoid10`
1. Load FRL data
2. Load Block demographics
3. Load Block demographics computed from student data (fill in before computing the demographics from the block data)

## FLR data
This data should be indexed by the column `Geoid10` as type `int64`.

We convert the `group` column into a more coherent index. In the original data there group ids is a integer from `1` to `353` for the blocks that are grouped together and the GEOID for the blocks that stand alone. For some reason the blocks that are grouped together only have `327` (not `353`) unique group indexes. Because of this, the max value of the new index is `3311` instead of `3285` (the actual length of the vector of unique group indexes).

In [ ]:
block_data_api = BlockDataApi()

frl_df = block_data_api.get_data(frl=True, user="juan").set_index('Geoid10')
frl_df.index.name = geoid_name
frl_df.columns = ['group', 'n', 'nFRL', 'nAALPI', 'nBoth']
frl_df['pctFRL'] = frl_df['nFRL'] / frl_df['n']
frl_df['pctAALPI'] = frl_df['nAALPI'] / frl_df['n']
frl_df['pctBoth'] = frl_df['nBoth'] / frl_df['n']

# TODO: What happens if 'pctBoth' = 'nBoth' / ('nFRL' + 'nAALPI' - 'nBoth')

# we want to find the blocks that share a group index
mask = frl_df['group'] < 1000
last_group_index = frl_df.loc[mask, 'group'].max()
# then we generate a new set of group indexes for the standalone blocks that is more coherent 
# with the indexes of the grouped blocks
num_of_new_indexes = np.sum(~mask)
new_group_index = np.arange(num_of_new_indexes) + 1 + last_group_index

frl_df.at[~mask, 'group'] = new_group_index
frl_df['group'] = frl_df['group'].astype('int64')
print(frl_df.shape)
frl_df.tail()

## Block Demographics

This data should be indexed by the column `Block` as type `int64`.

In [ ]:
demo_df = block_data_api.get_data().set_index('Block')['BlockGroup'].dropna()
demo_df.index.name = geoid_name
print(demo_df.shape)
print(demo_df.head())

## Student Demographics

This data should be indexed by the column `census_block` as type `int64`.

In [ ]:
periods_list = ["1415", "1516", "1617", "1718", "1819", "1920"]
student_data_api = StudentDataApi()

df_students = student_data_api.get_data(periods_list)
mask = df_students[_census_block_column] == 'NaN'
df_students.drop(df_students.index[mask], inplace=True)
df_students[geoid_name]=df_students['census_block'].astype('int64')

In [ ]:
def get_group_value(x):
    return x.iloc[0]

stud_df = df_students.groupby(geoid_name)[_diversity_index_features].agg(get_group_value)
print(stud_df.shape)
stud_df.head()

## Join data frames

In [ ]:
# TODO: check indexes

df = pd.concat([demo_df.to_frame(), stud_df.reindex(demo_df.index), frl_df.reindex(demo_df.index)],
               axis=1,
               ignore_index=False)
df.head()

# Creat map plots

In [ ]:
geodata_path = '/share/data/school_choice/dssg/census2010/'
file_name = 'geo_export_e77bce0b-6556-4358-b36b-36cfcf826a3c'
data_types = ['.shp', '.dbf', '.prj', '.shx']

sfusd_map = gpd.read_file(geodata_path + file_name + data_types[0])
sfusd_map[geoid_name] = sfusd_map['geoid10'].astype('int64')
sfusd_map.set_index(geoid_name, inplace=True)

In [ ]:
pct_cols = ['group', 'pctFRL', 'pctAALPI', 'pctBoth']
sfusd_map_df = pd.concat([sfusd_map.reindex(df.index), df[pct_cols]], axis=1, ignore_index=False)

In [ ]:
def plot_column(df_map, column, cmap="viridis"):

    fig, ax = plt.subplots(figsize=(30,30))
    
    if "Count" in column:
        cmap = "PRGn"
    elif "%" in column:
        cmap = "YlOrRd"
    
    df_map.plot(column=column, ax=ax, cmap=cmap, 
                         legend=True, legend_kwds={'orientation': "horizontal"},
                         missing_kwds={'color': 'lightgrey'})
    ax.set_title(column, fontsize=50)
    
    plt.show()

In [ ]:
plot_column(sfusd_map_df, 'pctFRL')

# Solve Knapsack

We are going to solve Knapsack problem for the block groups given by the `group` column.

In [ ]:
data = df[['n', 'nFRL', 'group']].groupby('group').agg(get_group_value)
target_group = 'nFRL'
data['nOther'] = data['n'] - data[target_group]
data.dropna(inplace=True)
data = data.round().astype('int64')
data.describe()

In [ ]:
from src.d04_modeling.knapsack_approx import KnapsackApprox

if False:
    solver = KnapsackApprox(eps=.5, data=data.copy(),
                            value_col=target_group,
                            weight_col='nOther',
                            scale=False)

    solver.solve()

In [ ]:
total_students = data['n'].sum()
fp_rate = 0.1
w_max = fp_rate * total_students
print(total_students, w_max)
if False:
    v_opt, solution_set = solver.get_solution(w_max=w_max)
    solution_set = pd.Index(solution_set, name=data.index.name)

In [ ]:
if False:
    solution_set.to_series().to_pickle('solution_set.pkl')

In [ ]:
solution_set = pd.read_pickle('solution_set.pkl')
solution_set = pd.Index(solution_set.values, name=geoid_name)

## Results

Lets see what our assignment looks like

In [ ]:
data['focal'] = 0
data.loc[solution_set, 'focal'] = 1
data.groupby('focal').sum()

We can observe that the number of `nOther` labeled as `focal = 1` is roughly 10 percent of the total students.

In [ ]:
def get_label(x, solution_set):
    if np.isfinite(x):
        return 1. if x in solution_set else 0.
    else:
        return np.nan

sfusd_map_df['focal'] = sfusd_map_df['group'].apply(lambda x: get_label(x, solution_set))

In [ ]:
plot_column(sfusd_map_df, 'focal')

# Train Logistic Regression

We now use the labeled data to train a logistic regression over the aggregated block data

## Model

We can use the contribution of each block group to their label as weights for the training process. Let each block $b$ be of type $y=k$, with $k\in \{0,1\}$, have $v_b$ focal students and $u_b$ non-focal students. Then we can define the weight of each block $w_b$ in the training process as

$$w_b = \frac{\mathbb{I}(y_b=1)v_b + \mathbb{I}(y_b=0)u_b}{\sum_b\mathbb{I}(y_b=1)v_b + \mathbb{I}(y_b=0)u_b}$$

First we should inspect what our new labeled data set looks like

In [ ]:
df_labeled = df.copy()
df_labeled['focal'] = df_labeled['group'].apply(lambda x: get_label(x, solution_set))
df_labeled.dropna(inplace=True)
df_labeled.head()

We should also see what each block group looks like. Do they have the same demographic characteristics? Maybe this is an analysis that should have been done previously? Should we have grouped the blocks under the column `'BlockGroup'` instead of an psudo-arbitrary assigment (what Joseph did).

In [ ]:
mask = df_labeled['group'] == 50
df_labeled.loc[mask]

It seems that blocks in the same `group` have different demographics. We are going to group and average the demographics. __Note__: _Since the counts are constant group counts, taking the average of these columns should yield the group counts. The same goes for the focal labels_

In [ ]:
data_glm = df_labeled.groupby('group').mean()
data_glm['focal'] = data_glm['focal'].astype('int64')

In [ ]:
data_glm.loc[50].to_frame().T

In [ ]:
data_glm['nOther'] = data_glm['n'] - data_glm[target_group]
summary = data_glm[['focal', 'n', 'nOther', target_group]].round().astype('int64')
summary = summary.groupby('focal').sum()
summary

In [ ]:
summary / summary['n'].sum()

In [ ]:
import statsmodels.api as sm

In [ ]:
glm_covariates = df_labeled[_diversity_index_features].astype('float64')
glm_covariates['const'] = 1.
glm_covariates.head()

In [ ]:
glm_response = df_labeled['focal']

In [ ]:
glm_binom = sm.GLM(glm_response, glm_covariates, family=sm.families.Binomial())
res = glm_binom.fit()
print(res.summary())